In [1]:
import json
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#### 1. Explore correlation coefficients and lasso regression to determine which factors are best for predicting retention.

#### 2. Create a Logistic Regression Model to predict rider retention

In [2]:
#location of data file
file_path = '../ultimate_challenge/ultimate_data_challenge.json'

In [3]:
json_data = json.load(open(file_path))
rider_df = pd.DataFrame(json_data)
rider_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null object
phone                     49604 non-null object
signup_date               50000 non-null object
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
ultimate_black_user       50000 non-null bool
weekday_pct               50000 non-null float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


### Clean Data

In [4]:
# convert last_trip_date and signup_date columns to datetime objects
rider_df.last_trip_date = pd.to_datetime(rider_df.last_trip_date)
rider_df.signup_date = pd.to_datetime(rider_df.signup_date)

# fill rows containing missing ratings
# compute average rating given by all drivers
mean_rating_by_driver = rider_df.avg_rating_by_driver.mean()

# fill average rating by driver with average rating of driver.
rider_df.avg_rating_by_driver = rider_df.avg_rating_by_driver.fillna(rider_df.avg_rating_of_driver)

# fill remaining missing values with oringinal mean of ratings given by drivers
rider_df.avg_rating_by_driver = rider_df.avg_rating_by_driver.fillna(mean_rating_by_driver)

#
rider_df.avg_rating_of_driver = rider_df.avg_rating_of_driver.fillna(rider_df.avg_rating_by_driver)

# if iPhone or Android not specified, insert "other"
rider_df.phone = rider_df.phone.fillna('other')

In [5]:
def rode_recently(last_ride):
    """
    Param: date of last activity
    Used with map() to compute if user was active in the last 30 days
    Returns: boolean (1 or 0)
    """
    current_day = rider_df.last_trip_date.max()
    last_30_days = current_day - timedelta(days=30)
    
    return (last_ride>=last_30_days)*1

In [6]:
# create column describing retention of users
rider_df['retained'] = rider_df.last_trip_date.map(rode_recently)

In [7]:
rider_df

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,ultimate_black_user,weekday_pct,retained
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2,1
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0,0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0,0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0,1
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.63,4.2,5.0,1.00,King's Landing,2014-06-05,iPhone,2014-01-25,0.0,0,False,100.0,1
49996,0.00,4.0,4.0,1.00,Astapor,2014-01-25,iPhone,2014-01-24,0.0,1,False,0.0,0
49997,3.86,5.0,5.0,1.00,Winterfell,2014-05-22,Android,2014-01-31,0.0,0,True,100.0,0
49998,4.58,3.5,3.0,1.00,Astapor,2014-01-15,iPhone,2014-01-14,0.0,2,False,100.0,0


In [8]:
# Fraction of observed users that were retained
print(rider_df.retained.mean())

0.37608


In [9]:
def replace_with_dummies(df, columns):
    """
    Takes a df and a list of categorial columns with discrete values
    Returns a df with categorical columns replaced with binary columns
    """
    for col in columns:
        temp_df= df[[col]]
        temp_df = pd.get_dummies(temp_df)
        df[temp_df.columns] = temp_df
        df = df.drop(col, axis=1)
    return df

In [10]:
# one hot encode categorical columns
rider_df = replace_with_dummies(rider_df, ['city', 'phone'])
rider_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      50000 non-null float64
avg_rating_of_driver      50000 non-null float64
avg_surge                 50000 non-null float64
last_trip_date            50000 non-null datetime64[ns]
signup_date               50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
ultimate_black_user       50000 non-null bool
weekday_pct               50000 non-null float64
retained                  50000 non-null int64
city_Astapor              50000 non-null uint8
city_King's Landing       50000 non-null uint8
city_Winterfell           50000 non-null uint8
phone_Android             50000 non-null uint8
phone_iPhone              50000 non-null uint8
phone_other               50000 non-null uint8
dtypes: bool(1), datetime64[ns](2), float64(6), int64(2), uint8

In [11]:
# define current day outside of the function for efficiency when mapping
current_day = rider_df.last_trip_date.max()
def days_since(date):
    """
    returns the number of days since a given date
    """
    return (current_day - date).days
    
# create 'days_since' columns
rider_df['days_since_last_trip'] = rider_df.last_trip_date.map(days_since)
rider_df['days_since_signup'] = rider_df.signup_date.map(days_since)

# compute number of days user has been active
rider_df['days_active'] = (rider_df.last_trip_date - rider_df.signup_date).map(lambda delta: delta.days)

# convert ultimate_black_user to numberic boolean column
rider_df.ultimate_black_user = rider_df.ultimate_black_user*1

# inspect the dataframe
rider_df


,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,last_trip_date,signup_date,surge_pct,trips_in_first_30_days,ultimate_black_user,weekday_pct,retained,city_Astapor,city_King's Landing,city_Winterfell,phone_Android,phone_iPhone,phone_other,days_since_last_trip,days_since_signup,days_active
0,3.67,5.0,4.7,1.10,2014-06-17,2014-01-25,15.4,4,1,46.2,1,0,1,0,0,1,0,14,157,143
1,8.26,5.0,5.0,1.00,2014-05-05,2014-01-29,0.0,0,0,50.0,0,1,0,0,1,0,0,57,153,96
2,0.77,5.0,4.3,1.00,2014-01-07,2014-01-06,0.0,3,0,100.0,0,1,0,0,0,1,0,175,176,1
3,2.36,4.9,4.6,1.14,2014-06-29,2014-01-10,20.0,9,1,80.0,1,0,1,0,0,1,0,2,172,170
4,3.13,4.9,4.4,1.19,2014-03-15,2014-01-27,11.8,14,0,82.4,0,0,0,1,1,0,0,108,155,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.63,4.2,5.0,1.00,2014-06-05,2014-01-25,0.0,0,0,100.0,1,0,1,0,0,1,0,26,157,131
49996,0.00,4.0,4.0,1.00,2014-01-25,2014-01-24,0.0,1,0,0.0,0,1,0,0,0,1,0,157,158,1
49997,3.86,5.0,5.0,1.00,2014-05-22,2014-01-31,0.0,0,1,100.0,0,0,0,1,1,0,0,40,151,111
49998,4.58,3.5,3.0,1.00,2014-01-15,2014-01-14,0.0,2,0,100.0,0,1,0,0,0,1,0,167,168,1


### Scale cleaned and formatted columns

In [12]:
rider_df

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,last_trip_date,signup_date,surge_pct,trips_in_first_30_days,ultimate_black_user,weekday_pct,retained,city_Astapor,city_King's Landing,city_Winterfell,phone_Android,phone_iPhone,phone_other,days_since_last_trip,days_since_signup,days_active
0,3.67,5.0,4.7,1.10,2014-06-17,2014-01-25,15.4,4,1,46.2,1,0,1,0,0,1,0,14,157,143
1,8.26,5.0,5.0,1.00,2014-05-05,2014-01-29,0.0,0,0,50.0,0,1,0,0,1,0,0,57,153,96
2,0.77,5.0,4.3,1.00,2014-01-07,2014-01-06,0.0,3,0,100.0,0,1,0,0,0,1,0,175,176,1
3,2.36,4.9,4.6,1.14,2014-06-29,2014-01-10,20.0,9,1,80.0,1,0,1,0,0,1,0,2,172,170
4,3.13,4.9,4.4,1.19,2014-03-15,2014-01-27,11.8,14,0,82.4,0,0,0,1,1,0,0,108,155,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.63,4.2,5.0,1.00,2014-06-05,2014-01-25,0.0,0,0,100.0,1,0,1,0,0,1,0,26,157,131
49996,0.00,4.0,4.0,1.00,2014-01-25,2014-01-24,0.0,1,0,0.0,0,1,0,0,0,1,0,157,158,1
49997,3.86,5.0,5.0,1.00,2014-05-22,2014-01-31,0.0,0,1,100.0,0,0,0,1,1,0,0,40,151,111
49998,4.58,3.5,3.0,1.00,2014-01-15,2014-01-14,0.0,2,0,100.0,0,1,0,0,0,1,0,167,168,1


In [13]:
# specify columns with 
continuous_features = ['avg_dist', 'avg_rating_by_driver', 'avg_rating_of_driver', 'avg_surge',
                      'surge_pct', 'trips_in_first_30_days']

In [14]:
def scale_column(df, col):
    """
    takes a dataframe and column
    returns a dataframe with the specified column scaled between 0,1
    """
    scaler = MinMaxScaler()
    df[col] = scaler.fit_transform(df[[col]])
    return df

# loop through continous variable columns than need scaling
#for column in continuous_features:
#    rider_df = scale_column(rider_df, column)
rider_df


,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,last_trip_date,signup_date,surge_pct,trips_in_first_30_days,ultimate_black_user,weekday_pct,retained,city_Astapor,city_King's Landing,city_Winterfell,phone_Android,phone_iPhone,phone_other,days_since_last_trip,days_since_signup,days_active
0,0.022801,1.000,0.925,0.014286,2014-06-17,2014-01-25,0.154,0.032,1,46.2,1,0,1,0,0,1,0,14,157,143
1,0.051317,1.000,1.000,0.000000,2014-05-05,2014-01-29,0.000,0.000,0,50.0,0,1,0,0,1,0,0,57,153,96
2,0.004784,1.000,0.825,0.000000,2014-01-07,2014-01-06,0.000,0.024,0,100.0,0,1,0,0,0,1,0,175,176,1
3,0.014662,0.975,0.900,0.020000,2014-06-29,2014-01-10,0.200,0.072,1,80.0,1,0,1,0,0,1,0,2,172,170
4,0.019446,0.975,0.850,0.027143,2014-03-15,2014-01-27,0.118,0.112,0,82.4,0,0,0,1,1,0,0,108,155,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.034978,0.800,1.000,0.000000,2014-06-05,2014-01-25,0.000,0.000,0,100.0,1,0,1,0,0,1,0,26,157,131
49996,0.000000,0.750,0.750,0.000000,2014-01-25,2014-01-24,0.000,0.008,0,0.0,0,1,0,0,0,1,0,157,158,1
49997,0.023981,1.000,1.000,0.000000,2014-05-22,2014-01-31,0.000,0.000,1,100.0,0,0,0,1,1,0,0,40,151,111
49998,0.028454,0.625,0.500,0.000000,2014-01-15,2014-01-14,0.000,0.016,0,100.0,0,1,0,0,0,1,0,167,168,1


In [15]:
int('stop here')

ValueError: invalid literal for int() with base 10: 'stop here'

In [19]:
from sklearn.preprocessing import scale

# specify feature columns
features = ['avg_dist', 'avg_rating_by_driver', 'avg_rating_of_driver', 'avg_surge',
       'surge_pct', 'trips_in_first_30_days',
       'ultimate_black_user', 'weekday_pct', 'city_Astapor',
       "city_King's Landing", 'city_Winterfell', 'phone_Android',
       'phone_iPhone', 'phone_other', 'days_since_last_trip',
       'days_since_signup', 'days_active']

# separate predictors and targets
predictors = rider_df[features].values
targets = (rider_df.retained).values

In [20]:
predictors = scale(predictors)

In [30]:
# Split the data into a training and test set.
X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=0.25, random_state=5)

# instantiate an L.R. Classifier
clf = LogisticRegression(solver='lbfgs')

# Fit the model on the trainng data.
clf.fit(X_train, y_train)

# predict on test set
predictions = clf.predict(X_test)

# Print the accuracy from the testing data.
print(accuracy_score(predictions, y_test))

0.99536


In [23]:
rider_df.retained.mean()

0.37608

### Create AUROC graph and find other meaningful methods of scoring. We are interested in targeting users that dropped the service, but were predicted to retain (False Positives). Find the best way to score this

### Use Lasso / Ridge Regression to identify feature significance and predictive power. 